In [1]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt
from ir_measures import *
import pandas as pd

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#pt.init()
ensure_pyterrier_is_loaded()
tira = Client()


PyTerrier 0.10.1 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [3]:
pt_dataset = pt.get_dataset("irds:ir-lab-padua-2024/longeval-2023-01-20240426-training")
pt_dataset_jun = pt.get_dataset("irds:ir-lab-padua-2024/longeval-2023-06-20240418-training")
pt_dataset_aug = pt.get_dataset("irds:ir-lab-padua-2024/longeval-2023-08-20240418-training")

In [4]:
bm25 = tira.pt.from_submission('ir-benchmarks/tira-ir-starter/BM25 Re-Rank (tira-ir-starter-pyterrier)', 'ir-benchmarks/longeval-2023-01-20240423-training')
sce = tira.pt.from_submission('ir-benchmarks/fschlatt/sparse-cross-encoder-4-512', 'ir-benchmarks/longeval-2023-01-20240423-training')
cb = tira.pt.from_submission('ir-benchmarks/tira-ir-starter/ColBERT Re-Rank (tira-ir-starter-pyterrier)', 'ir-benchmarks/longeval-2023-01-20240423-training')
rz = tira.pt.from_submission('workshop-on-open-web-search/fschlatt/rank-zephyr', 'ir-benchmarks/longeval-2023-01-20240423-training')
fusion_jan = pt.transformer.get_transformer(pt.io.read_results('galapagos-tortoise-wsum.lag1'))
fusion_jun = pt.transformer.get_transformer(pt.io.read_results('galapagos-tortoise-wsum.lag6'))
fusion_aug = pt.transformer.get_transformer(pt.io.read_results('galapagos-tortoise-wsum.lag8'))

In [13]:
hypo_jan = pt.Experiment(
    [
        bm25,
        sce,
        cb,
        rz,
        fusion_jan,
    ],
    pt_dataset.get_topics('text'),
    pt_dataset.get_qrels(),
    names=[
        'BM25', 
        "Sparse Cross-Encoder",
        'ColBERT',
        'RankZephyr', 
        'WSum',
        # "3-max-avg",
    ],
    eval_metrics=[nDCG(judged_only=True)@10, nDCG(judged_only=True), nDCG@10, nDCG],
    baseline = 4
)

hypo_jan

,name,nDCG(judged_only=True),nDCG(judged_only=True)@10,nDCG,nDCG@10,nDCG(judged_only=True) +,nDCG(judged_only=True) -,nDCG(judged_only=True) p-value,nDCG(judged_only=True)@10 +,nDCG(judged_only=True)@10 -,nDCG(judged_only=True)@10 p-value,nDCG +,nDCG -,nDCG p-value,nDCG@10 +,nDCG@10 -,nDCG@10 p-value
0,BM25,0.472638,0.444411,0.317285,0.199380,174.0,236.0,0.011915,179.0,230.0,0.006911,147.0,355.0,1.841320e-19,135.0,315.0,1.341727e-17
1,Sparse Cross-Encoder,0.474784,0.448109,0.336719,0.220869,181.0,228.0,0.031730,184.0,225.0,0.052011,183.0,314.0,5.228196e-08,153.0,287.0,7.902821e-10
2,ColBERT,0.476047,0.450091,0.330096,0.216457,194.0,221.0,0.111510,195.0,220.0,0.187790,175.0,326.0,4.984377e-11,160.0,287.0,2.137292e-10
3,RankZephyr,0.482076,0.455142,0.352920,0.246549,141.0,139.0,0.366029,135.0,135.0,0.775121,193.0,285.0,2.592030e-01,162.0,200.0,7.013943e-02
4,WSum,0.480725,0.454667,0.355077,0.251512,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
bm25_jun = tira.pt.from_submission('ir-benchmarks/tira-ir-starter/BM25 Re-Rank (tira-ir-starter-pyterrier)', 'ir-benchmarks/longeval-2023-06-20240418-training')
sce_jun = tira.pt.from_submission('ir-benchmarks/fschlatt/sparse-cross-encoder-4-512', 'ir-benchmarks/longeval-2023-06-20240418-training')
cb_jun = tira.pt.from_submission('ir-benchmarks/tira-ir-starter/ColBERT Re-Rank (tira-ir-starter-pyterrier)', 'ir-benchmarks/longeval-2023-06-20240418-training')
rz_jun = tira.pt.from_submission('workshop-on-open-web-search/fschlatt/rank-zephyr', 'ir-benchmarks/longeval-2023-06-20240418-training')

### Hypothesis Test on June-Dataset.

In [7]:
hypo_jun = pt.Experiment(
    [
        bm25_jun,
        sce_jun,
        cb_jun,
        rz_jun,
        fusion_jun,
    ],
    pt_dataset_jun.get_topics('text'),
    pt_dataset_jun.get_qrels(),
    #pd.read_csv("01_qrels_jun.txt"),
    names=[
        'BM25', 
        "Sparse Cross-Encoder",
        'ColBERT',
        'RankZephyr', 
        'WSum',
    ],
    eval_metrics=[nDCG(judged_only=True)@10, nDCG(judged_only=True), nDCG@10, nDCG],
    baseline = 4
)

hypo_jun


,name,nDCG(judged_only=True),nDCG(judged_only=True)@10,nDCG,nDCG@10,nDCG(judged_only=True) +,nDCG(judged_only=True) -,nDCG(judged_only=True) p-value,nDCG(judged_only=True)@10 +,nDCG(judged_only=True)@10 -,nDCG(judged_only=True)@10 p-value,nDCG +,nDCG -,nDCG p-value,nDCG@10 +,nDCG@10 -,nDCG@10 p-value
0,BM25,0.387891,0.375125,0.252253,0.165902,95.0,154.0,0.001333,96.0,153.0,0.001136,97.0,234.0,3.387580e-17,74.0,213.0,1.294152e-17
1,Sparse Cross-Encoder,0.393611,0.381502,0.276560,0.201901,101.0,145.0,0.038635,96.0,149.0,0.034443,111.0,220.0,1.356805e-04,100.0,193.0,1.761921e-05
2,ColBERT,0.388463,0.376752,0.263776,0.183421,106.0,149.0,0.002078,104.0,151.0,0.003803,95.0,236.0,1.201873e-08,84.0,205.0,9.834654e-11
3,RankZephyr,0.400839,0.390550,0.294686,0.227905,84.0,86.0,0.615196,84.0,83.0,0.297015,135.0,176.0,3.441785e-01,118.0,120.0,9.810752e-01
4,WSum,0.399975,0.388633,0.292732,0.227834,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
bm25_aug = tira.pt.from_submission('ir-benchmarks/tira-ir-starter/BM25 Re-Rank (tira-ir-starter-pyterrier)', 'ir-benchmarks/longeval-2023-08-20240418-training')
sce_aug = tira.pt.from_submission('ir-benchmarks/fschlatt/sparse-cross-encoder-4-512', 'ir-benchmarks/longeval-2023-08-20240418-training')
cb_aug = tira.pt.from_submission('ir-benchmarks/tira-ir-starter/ColBERT Re-Rank (tira-ir-starter-pyterrier)', 'ir-benchmarks/longeval-2023-08-20240418-training')
rz_aug = tira.pt.from_submission('workshop-on-open-web-search/fschlatt/rank-zephyr', 'ir-benchmarks/longeval-2023-08-20240418-training')

### Hypothesis Test on August-Dataset.

In [9]:
hypo_aug = pt.Experiment(
    [
        bm25_aug,
        sce_aug,
        cb_aug,
        rz_aug,
        fusion_aug,
    ],
    pt_dataset_aug.get_topics('text'),
    pt_dataset_aug.get_qrels(),
    names=[
        'BM25', 
        "Sparse Cross-Encoder",
        'ColBERT',
        'RankZephyr', 
        'WSum',
    ],
    eval_metrics=[nDCG(judged_only=True)@10, nDCG(judged_only=True), nDCG@10, nDCG],
    baseline = 4
)

hypo_aug

,name,nDCG(judged_only=True),nDCG(judged_only=True)@10,nDCG,nDCG@10,nDCG(judged_only=True) +,nDCG(judged_only=True) -,nDCG(judged_only=True) p-value,nDCG(judged_only=True)@10 +,nDCG(judged_only=True)@10 -,nDCG(judged_only=True)@10 p-value,nDCG +,nDCG -,nDCG p-value,nDCG@10 +,nDCG@10 -,nDCG@10 p-value
0,BM25,0.300187,0.296796,0.191457,0.140883,307.0,409.0,0.000034,307.0,403.0,0.000013,347.0,771.0,2.643216e-34,283.0,656.0,4.964397e-35
1,Sparse Cross-Encoder,0.305075,0.302068,0.211935,0.168767,329.0,358.0,0.375969,325.0,359.0,0.297648,447.0,654.0,2.137901e-05,379.0,547.0,8.867774e-06
2,ColBERT,0.305006,0.301849,0.205953,0.161361,346.0,384.0,0.412428,342.0,388.0,0.290758,416.0,676.0,3.051411e-11,361.0,546.0,2.981866e-11
3,RankZephyr,0.306502,0.303919,0.219348,0.177775,219.0,219.0,0.740464,213.0,218.0,0.655888,418.0,576.0,5.159930e-01,343.0,376.0,1.510246e-01
4,WSum,0.306251,0.303571,0.220011,0.179789,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Compute differences in nDCG.

In [12]:
df = pd.DataFrame({'nDCG_01-06': [0,0,0,0,0],
                   'nDCG_06-08': [0,0,0,0,0],
                   'nDCG_01-08': [0,0,0,0,0],
                   'nDCG@10_01-06': [0,0,0,0,0],
                   'nDCG@10_06-08': [0,0,0,0,0],
                   'nDCG@10_01-08': [0,0,0,0,0],})
df["nDCG_01-06"] = hypo_jan["nDCG"] - hypo_jun["nDCG"]
df["nDCG_06-08"] = hypo_jun["nDCG"] - hypo_aug["nDCG"]
df["nDCG_01-08"] = hypo_jan["nDCG"] - hypo_aug["nDCG"]
df["nDCG@10_01-06"] = hypo_jan["nDCG@10"] - hypo_jun["nDCG@10"]
df["nDCG@10_06-08"] = hypo_jun["nDCG@10"] - hypo_aug["nDCG@10"]
df["nDCG@10_01-08"] = hypo_jan["nDCG@10"] - hypo_aug["nDCG@10"]

df

,nDCG_01-06,nDCG_06-08,nDCG_01-08,nDCG@10_01-06,nDCG@10_06-08,nDCG@10_01-08
0,0.065032,0.060796,0.125828,0.033478,0.025019,0.058497
1,0.060159,0.064626,0.124784,0.018968,0.033134,0.052102
2,0.066320,0.057823,0.124142,0.033036,0.022060,0.055095
3,0.058234,0.075338,0.133571,0.018644,0.050129,0.068773
4,0.062345,0.072722,0.135067,0.023678,0.048044,0.071723
